In [3]:
import pandas as pd #data processing
import numpy as np #maths
import sklearn
import requests as req
import gzip

In [6]:
# URL of the IMDB datasets
url_title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
url_crew = 'https://datasets.imdbws.com/title.crew.tsv.gz'
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

# Download the file from the URL
res_title_basics = req.get(url_title_basics).content
res_crew = req.get(url_crew).content
res_ratings = req.get(url_ratings).content

title_basics = gzip.decompress(res_title_basics)
title_crew = gzip.decompress(res_crew)
title_ratings = gzip.decompress(res_ratings)

done
